In [10]:
from ultralytics import YOLO
import cv2
import os

# Load your model
model = YOLO("/Users/secret/my_project/garbage_detection/best (3).pt")

def split_image_into_tiles(image, tile_size=512):
    tiles = []
    h, w, _ = image.shape
    for y in range(0, h, tile_size):
        for x in range(0, w, tile_size):
            tile = image[y:y+tile_size, x:x+tile_size]
            tiles.append((tile, (x, y)))
    return tiles

def detect_on_tiles(model, tiles):
    all_results = []
    total_garbage = 0
    total_garbage_area = 0  # To track total area of garbage

    for tile, (x, y) in tiles:
        temp_tile_path = f'temp_tile_{x}_{y}.jpg'
        cv2.imwrite(temp_tile_path, tile)
        
        results = model.predict(temp_tile_path, conf=0.4, verbose=False)
        all_results.append((results[0], x, y))
        
        # Calculate area of each detection
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            width = x2 - x1
            height = y2 - y1
            total_garbage_area += width * height
            
        garbage_count = len(results[0].boxes)
        total_garbage += garbage_count
        
        os.remove(temp_tile_path)

    return all_results, total_garbage, total_garbage_area

def calculate_cleanliness_score(image, total_garbage_area):
    # Calculate coverage percentage
    h, w = image.shape[:2]
    total_pixels = h * w
    coverage_percent = (total_garbage_area / total_pixels) * 100
    
    # Convert to 0-10 scale (10 = clean, 0 = completely dirty)
    score = max(0, 10 - (coverage_percent / 10))  # Deduct 0.1 points per 1% coverage
    return round(score, 1)

# Main code
img_path = "/Users/secret/my_project/garbage_detection/142980439-many-garbage-and-dirty-streets-after-holy-festival-hari-raya-aidilfitri-ramzan-ramadan-in-kuala.jpg"

# Load image
image = cv2.imread(img_path)

if image is None:
    print("Error: Could not load image")
else:
    # Split into tiles
    tiles = split_image_into_tiles(image, tile_size=512)
    
    # Detect garbage
    all_results, total_garbage, total_area = detect_on_tiles(model, tiles)
    
    # Calculate cleanliness score
    cleanliness_score = calculate_cleanliness_score(image, total_area)
    
    print(f"🗑️ Total garbage items detected: {total_garbage}")
    print(f"⭐ Cleanliness Score: {cleanliness_score}/10")
    
    # Draw and display results
    output_image = draw_detections_on_image(image, all_results)
    cv2.imwrite('final_result.jpg', output_image)
    cv2.imshow('Detected Garbage', output_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

🗑️ Total garbage items detected: 5
⭐ Cleanliness Score: 8.2/10


In [5]:
import im_show

ModuleNotFoundError: No module named 'im_show'